# generate chars using bigram

In [49]:
from google.colab import drive # access google drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


##import library

In [50]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


##define hyperparameter

In [51]:
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

##Loading data

In [52]:
wizard_of_oz_address =  "/content/gdrive/MyDrive/"+'Colab Notebooks'+"/LLM/wizard_of_oz.txt"

In [53]:
with open (wizard_of_oz_address,'r',encoding='utf-8') as f:
  text = f.read()
chars = sorted(set(text))
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [54]:
vocab_size = len(chars)
print(vocab_size)

81


##Encoder-Decoder

In [55]:
from sys import int_info
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s :[string_to_int[c] for c in s]
decode = lambda l : ''.join([int_to_string[i] for i in l])
data = torch.tensor(encode(text),dtype=torch.long)

##data batch

In [56]:
number_div = int(0.8*len(data))
train_data = data[:number_div]
val_data = data[number_div:]

def get_batch(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1]for i in ix])
  x,y = x.to(device) , y.to(device)
  return x,y

x,y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[69, 54, 73, 61,  1, 54, 56, 71],
        [70, 74, 62, 71, 58, 57,  1, 73],
        [43,  1, 44, 32, 29,  1, 37, 25],
        [56, 54, 65, 66,  1, 75, 68, 62]], device='cuda:0')
targets:
tensor([[54, 73, 61,  1, 54, 56, 71, 68],
        [74, 62, 71, 58, 57,  1, 73, 61],
        [ 1, 44, 32, 29,  1, 37, 25, 31],
        [54, 65, 66,  1, 75, 68, 62, 56]], device='cuda:0')


##model

In [57]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train','val']:
    losses = torch.zeros (eval_iters)
    for k in range(eval_iters):
      X, Y =  get_batch(split)
      logits, loss = model (X,Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [58]:
token_embedding_table = nn.Embedding(vocab_size, vocab_size)

print( token_embedding_table )

Embedding(81, 81)


In [75]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
      super().__init__()
      self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        #print("logits",logits.shape)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        print("index : ",index)
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1 , :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

index :  tensor([[0]], device='cuda:0')

NCP9w7WzOo5xX[;yq)RThZC_Rv JE
G(e;(5zdJ"L,V9zd*GfGKh1dgTh35﻿ryk!Vn(vR7)2Pm0&KyFLEt54giAp.N_T'9.*J2Ss(y'B"DWini?l6jxm1zdfiOO?Hs&.T).OHo_52hU:ba-p.h?"[k!uMfJmq-Gh4
v.TxUQLmJdg lp_J(35B)Vh1f]y5Y]7X899FWi&PC[pj3]nHmTPbTHmf&.s[8i-Rgz!1(&Ri.Y',(IloLtaR8)Xn _pwHUv 7KAz0whW
Gh:tWJP lv﻿hhwS(,s)Lfa4z;J-Fcoc4P*!tk1dQ0JZDgdrVAp]﻿Byd&dhX'g2LGw8f_xi.Husf]2Pcr]b3pVF3l,82*UnC8sZwt'4p&hV)sdfNaAtxN&[A*﻿&WA.,e6tvD
RR9ByM5a!nNQWNJeY_MFL]f]﻿(6"4﻿2T,ar!!-v&J'xSitO*QNanN7hUWinI4tm2Z0Ggbh3LE)?]bTv&Kk1fg25:V[-:I:Iq?fafl


##train

In [70]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits torch.Size([4, 8, 81])
logits tor

##generate char

In [71]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

index :  tensor([[0]], device='cuda:0')
logits torch.Size([1, 1, 81])
logits torch.Size([1, 2, 81])
logits torch.Size([1, 3, 81])
logits torch.Size([1, 4, 81])
logits torch.Size([1, 5, 81])
logits torch.Size([1, 6, 81])
logits torch.Size([1, 7, 81])
logits torch.Size([1, 8, 81])
logits torch.Size([1, 9, 81])
logits torch.Size([1, 10, 81])
logits torch.Size([1, 11, 81])
logits torch.Size([1, 12, 81])
logits torch.Size([1, 13, 81])
logits torch.Size([1, 14, 81])
logits torch.Size([1, 15, 81])
logits torch.Size([1, 16, 81])
logits torch.Size([1, 17, 81])
logits torch.Size([1, 18, 81])
logits torch.Size([1, 19, 81])
logits torch.Size([1, 20, 81])
logits torch.Size([1, 21, 81])
logits torch.Size([1, 22, 81])
logits torch.Size([1, 23, 81])
logits torch.Size([1, 24, 81])
logits torch.Size([1, 25, 81])
logits torch.Size([1, 26, 81])
logits torch.Size([1, 27, 81])
logits torch.Size([1, 28, 81])
logits torch.Size([1, 29, 81])
logits torch.Size([1, 30, 81])
logits torch.Size([1, 31, 81])
logits t